In [ ]:
import pandas as pd
import numpy as np

In [2]:
imageMeasurements = '/Volumes/Aorta/aorta/results/general/numberNuc_Area_length_circumFINAL.csv'
microCT = '/Volumes/Aorta/aorta/results/microCT/totalLengthAortas.csv'
cellShapePath = '/Volumes/Aorta/aorta/results/cellSizeStatistics/allCellShapesUpdate.csv'

In [3]:
dfImage = pd.read_csv(imageMeasurements)
dfMicroCT = pd.read_csv(microCT)

get average length from images and microCT length to calculate the microCT_length factor for each aorta

In [ ]:
averages_by_ageImage = dfImage.groupby('age')['length'].transform('mean')
averages_by_ageMicroCT = dfMicroCT.groupby('age')['length_1_8'].mean()
averages_by_ageMicroCT = averages_by_ageMicroCT.reset_index()
dfImage['factorAorta'] = dfImage['length'] / averages_by_ageImage
merged_df = dfImage.merge(averages_by_ageMicroCT, on='age', suffixes=('_image', '_microCT'))

transform length to estimated microCT length and microCT-adjusted area

In [ ]:
merged_df['microCTLengthAdjusted'] = merged_df['factorAorta'] * merged_df['length_1_8'] * 1000
merged_df['areaMicroCTAdjusted'] = merged_df['microCTLengthAdjusted'] * merged_df['circumference']

calculate minor/major axis and area in micrometers/micrometers^2

In [ ]:
dfCellShape = pd.read_csv(cellShapePath)
#dCalibration: pixel to micrometers
dCalibration = 0.8599901168205362 

In [7]:
dfCellShape['major_axis_length_Micro'] = dfCellShape['major_axis_length']  * dCalibration
dfCellShape['minor_axis_length_Micro'] = dfCellShape['minor_axis_length']  * dCalibration
dfCellShape['area_Micro'] = dfCellShape['area']  * dCalibration * dCalibration

apply microCT adjustment to major axis and cell area

In [ ]:
dfCellShape['areaAdjusted'] = dfCellShape['major_axis_length_Micro'] * dfCellShape['minor_axis_length_Micro'] * 0.5
dfCellShape['areafraction'] = dfCellShape['areaAdjusted']/dfCellShape['area_Micro']
merged_dfFilter = merged_df[['age', 'aorta', 'length', 'microCTLengthAdjusted']]
merged_dfCellShape = dfCellShape.merge(merged_dfFilter, on=['age', 'aorta'])
merged_dfCellShape['major_axis_length_MicroCT'] = dfCellShape['major_axis_length_Micro'] * (merged_dfCellShape['microCTLengthAdjusted']/merged_dfCellShape['length'])

merged_dfCellShape['areaAdjusted'] = merged_dfCellShape['factorArea'].mean() * merged_dfCellShape['minor_axis_length_Micro']*merged_dfCellShape['major_axis_length_Micro']
merged_dfCellShape['areaAdjusted_MicroCT'] = merged_dfCellShape['factorArea'].mean() * merged_dfCellShape['minor_axis_length_Micro']*merged_dfCellShape['major_axis_length_MicroCT']

merged_dfCellShape['aspectRatio'] = merged_dfCellShape['major_axis_length_MicroCT']/df['minor_axis_length_Micro']
merged_dfCellShape['factorArea'] = merged_dfCellShape['area']/(merged_dfCellShape['minor_axis_length']*merged_dfCellShape['major_axis_length'])

save dataframe

In [9]:
merged_dfCellShape.to_csv('/Volumes/Aorta/aorta/results/cellSizeStatistics/allCellShapesUpdate_MicroCT.csv')
merged_df.to_csv('/Volumes/Aorta/aorta/results/general/numberNuc_Area_length_circumFINAL_MicroCT.csv')